In [1]:
import sys
sys.path.append('../')

from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
import os
import openai


In [2]:
llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)
llm._generate("Tell me a joke")

LLMResult(generations=[[Generation(text='\n\nWhy did the chicken cross the road?\n\nTo get to the other side.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\nWhy did the chicken cross the road?\n\nTo get to the other side.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [], [], [], [], [], [], [], [], [], [], [], [], []], llm_output={'token_usage': {'prompt_tokens': 4, 'completion_tokens': 38, 'total_tokens': 42}})

In [7]:
llm.generate("Tell me a joke")

LLMResult(generations=[[Generation(text='\n\nWhy did the chicken cross the road?\n\nTo get to the other side!', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\nWhy did the chicken cross the road?\n\nTo get to the other side!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [], [], [], [], [], [], [], [], [], [], [], [], []], llm_output={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 4, 'total_tokens': 42}})

In [8]:
llm = OpenAI(temperature=0) 
llm.generate("Tell me a joke")


LLMResult(generations=[[Generation(text='\n\nQ: What did the fish say when it hit the wall?\nA: Dam!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [], [], [], [], [], [], [], [], [], [], [], [], []], llm_output={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 4, 'total_tokens': 24}})

In [ ]:
llm.generate

In [2]:

llm = OpenAI(temperature=0)   #,openai_api_key=openai_api_key)
search = SerpAPIWrapper() #serpapi_api_key=serpapi_api_key
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:////home/rubensmau/luby/langchain/Chinook.db") 
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

In [5]:
mrkl = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True,return_intermediate_steps=True)

In [6]:
mrkl.agent

ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=SharedCallbackManager(ignore_llm=False, ignore_chain=False, ignore_agent=False), verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events. You should ask targeted questions\nCalculator: useful for when you need to answer questions about math\nFooBar DB: useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator, FooBar DB]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation c

In [7]:
res = mrkl.agent.llm_chain.prompt.template
print(res)

Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions
Calculator: useful for when you need to answer questions about math
FooBar DB: useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator, FooBar DB]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [3]:
llm.generate(["foo", "bar", "foo"])

LLMResult(generations=[[Generation(text='.bar\n\nFoo.bar is a common phrase used to refer to something that is not easily understood or is confusing. It can also be used to describe something that is difficult to explain or comprehend.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='-nav">\n            <li class="nav-item">\n              <a class="nav-link" href="index.php">Home</a>\n            </li>\n            <li class="nav-item">\n              <a class="nav-link" href="about.php">About</a>\n            </li>\n            <li class="nav-item">\n              <a class="nav-link" href="contact.php">Contact</a>\n            </li>\n            <li class="nav-item">\n              <a class="nav-link" href="login.php">Login</a>\n            </li>\n            <li class="nav-item">\n              <a class="nav-link" href="register.php">Register</a>\n            </li>\n          </ul>\n        </div>\n      </div>\n    </nav>\n\n    <!-- Page Content -

In [15]:
llm.generate("Who is Olivia Wilde's boyfriend?",stop='\n')

ServiceUnavailableError: The server is overloaded or not ready yet.

In [9]:
response = mrkl("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")




> Entering new AgentExecutor chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Who is Olivia Wilde's boyfriend?"
Observation: Harry Styles
Thought: I need to find out Harry Styles' age
Action: Search
Action Input: "How old is Harry Styles?"
Observation: 28 years
Thought: I need to calculate 28 raised to the 0.23 power
Action: Calculator
Action Input: 28^0.23

> Entering new LLMMathChain chain...
28^0.23

ServiceUnavailableError: The server is overloaded or not ready yet.

In [ ]:
response

In [ ]:
import json
print(json.dumps(response["intermediate_steps"], indent=2))

NameError: name 'response' is not defined

In [28]:
mrkl.run("What is Prince Harry of England's wife age multiplied to the prince´s age")



> Entering new AgentExecutor chain...
 I need to find out the ages of Prince Harry and his wife
Action: Search
Action Input: "Prince Harry age"
Observation: 38 years
Thought: I need to find out the age of Prince Harry's wife
Action: Search
Action Input: "Meghan Markle age"
Observation: 41 years
Thought: I now know the ages of both Prince Harry and his wife
Action: Calculator
Action Input: 38 * 41

> Entering new LLMMathChain chain...
38 * 41
Answer: 1558
> Finished LLMMathChain chain.

Observation: Answer: 1558
Thought: I now know the final answer
Final Answer: 1558
> Finished AgentExecutor chain.


'1558'

In [29]:
mrkl.run("What is the full name of the artist who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?")



> Entering new AgentExecutor chain...
 I need to find out the artist's full name and then search the FooBar database for their albums.
Action: Search
Action Input: "The Storm Before the Calm" artist
Observation: Alanis Morissette - the storm before the calm - Amazon.com Music.
Thought: I now need to search the FooBar database for Alanis Morissette's albums.
Action: FooBar DB
Action Input: What albums of Alanis Morissette are in the FooBar database?

> Entering new SQLDatabaseChain chain...
What albums of Alanis Morissette are in the FooBar database? 
SQLQuery: SELECT Title FROM Album INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = 'Alanis Morissette';
SQLResult: [('Jagged Little Pill',)]
Answer: The album 'Jagged Little Pill' by Alanis Morissette is in the FooBar database.
> Finished SQLDatabaseChain chain.

Observation:  The album 'Jagged Little Pill' by Alanis Morissette is in the FooBar database.
Thought: I now know the final answer.
Final Answer: Alanis 

"Alanis Morissette is the artist who recently released an album called 'The Storm Before the Calm' and the album 'Jagged Little Pill' by Alanis Morissette is in the FooBar database."

In [48]:
mrkl.run("Retrieve from FooBar artist database for a name beginning with Alanis, and then search her birthdate")



> Entering new AgentExecutor chain...
 I need to use FooBar DB to get the artist name, then use Search to get the birthdate
Action: FooBar DB
Action Input: What is the name of an artist in the FooBar database beginning with Alanis?

> Entering new SQLDatabaseChain chain...
What is the name of an artist in the FooBar database beginning with Alanis? 
SQLQuery: SELECT Name FROM Artist WHERE Name LIKE 'Alanis%';
SQLResult: [('Alanis Morissette',)]
Answer: Alanis Morissette
> Finished SQLDatabaseChain chain.

Observation:  Alanis Morissette
Thought: Now I need to use Search to get the birthdate
Action: Search
Action Input: When was Alanis Morissette born?
Observation: June 1, 1974
Thought: I now know the final answer
Final Answer: Alanis Morissette was born on June 1, 1974.
> Finished AgentExecutor chain.


'Alanis Morissette was born on June 1, 1974.'

mrkl.run("What albums of The Black Crowes are in the FooBar database? ")

In [42]:
mrkl.run("Search me 4 band names from Atlanta,USA. Then, check in the FooBar database their albums, for each of these band names")



> Entering new AgentExecutor chain...
 I need to search for the band names first, then check the FooBar database for their albums
Action: Search
Action Input: 4 band names from Atlanta, USA
Observation: In the early 1980s, Atlanta was the home of a thriving pop and new wave music scene featuring such bands as The Fans, The Brains, The Producers, The Raves, Baby and the Pacifiers, The Razor Boys, The Neuz, Desperate Angel, Samurai Catfish, Heathen Girls and Face of Concern.
Thought: Now I need to check the FooBar database for their albums
Action: FooBar DB
Action Input: What albums were released by The Fans, The Brains, The Producers, and The Raves?

> Entering new SQLDatabaseChain chain...
What albums were released by The Fans, The Brains, The Producers, and The Raves? 
SQLQuery: SELECT Album.Title FROM Album INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name IN ('The Fans', 'The Brains', 'The Producers', 'The Raves');
SQLResult: []
Answer: No albums were release

'No albums were released by The Fans, The Brains, The Producers, and The Raves.'